In [3]:
%run testcong.py
%run XE7

### Read in (or compute) the has table of $a_p\pmod{7}$ for $30$ primes over $400000$.  Computing this takes about 45 minutes

In [4]:
try:
    hashtab7_30 = load('hashtab7_30')
except IOError:
    hashtab7_30 = make_hash(7,11,400000,30)    
    hashtab7_30 = dict([(k,v) for k,v in hashtab7_30.items() if len(v)>1])
    save(hashtab7_30, 'hashtab7_30')
len(hashtab7_30)    

20138

### For each set of size ${}>1$, use Kraus's criterion to prove that the curves really are $7$-congruent.  Note that  we only test $p<10^7$ even if Kraus needs more, so the cases where a warning is output should be run again (this was done once outside this notebook).  NB This takes a long time.

In [3]:
bad_pairs = []
for s in hashtab7_30.values():
     if len(s)>1:
         E1 = EllipticCurve(s[0])
         for r in s[1:]:
             E2 = EllipticCurve(r)
             res, info = test_cong(7,E1,E2, mumax=10^7)
             if not res:
                 report(res,info,7,s[0],r)
                 bad_pairs.append([s[0],r])


<string>:105: RuntimeWarning: cypari2 leaked 1400 bytes on the PARI stack


KeyboardInterrupt: 

## The result of the previous cell is that there is just one bad pair:

In [5]:
#bad_pairs
bad_pairs = [['25921a1', '78400gw1']] # previous cell takes ages; this is the output

## We extract all the sets of size greater than 1:

In [6]:
isom_sets = [s for s in hashtab7_30.values() if len(s)>1]
len(isom_sets)

20138

## Separate out the reducible and irreducible sets:

In [7]:
isom_sets_red = []
isom_sets_irred = []
for s in isom_sets:
    isom_sets_irred.append(s) if test_irred(s) else isom_sets_red.append(s)
print("{} irreducible sets, {} reducible sets".format(len(isom_sets_irred),len(isom_sets_red)))

19883 irreducible sets, 255 reducible sets


## The long test found that 25921a is not actually 7-congruent to 78400gw1, so we remove it from that set of 4 to leave a set of 3

In [8]:
bad_s = [s for s in isom_sets_red if '25921a1' in s][0]
bad_s

['25921a1', '78400gw1', '336973z1', '336973ba1']

## Check that 3 of the 4 curves in this set really are congruent:

In [9]:
print("Testing curves in {} for congruence".format(bad_s))
E0 = EllipticCurve(bad_s[0])
#print("{}: {}".format(E0.label(), E0))
E1 = EllipticCurve(bad_s[1])
#print("{}: {}".format(E1.label(), E1))
E2 = EllipticCurve(bad_s[2])
#print("{}: {}".format(E2.label(), E2))
E3 = EllipticCurve(bad_s[3])
#print("{}: {}".format(E3.label(), E3))
verb=False
res, info = test_cong(7,E0,E1, mumax=10^7, verbose=verb)
assert not res
print("{} and {} are not congruent mod 7: {}".format(bad_s[0],bad_s[1],info))
assert test_cong(7,E0,E2, mumax=10^7, verbose=verb)
print("{} and {} are congruent mod 7".format(bad_s[0],bad_s[2]))
assert test_cong(7,E0,E3, mumax=10^7, verbose=verb)
print("{} and {} are congruent mod 7".format(bad_s[0],bad_s[3]))
#assert test_cong(7,E2,E3, mumax=10^7, verbose=verb)
#print("{} and {} are congruent mod 7".format(bad_s[2],bad_s[3]))

Testing curves in ['25921a1', '78400gw1', '336973z1', '336973ba1'] for congruence
25921a1 and 78400gw1 are not congruent mod 7: (29, 2, -2)
25921a1 and 336973z1 are congruent mod 7
25921a1 and 336973ba1 are congruent mod 7


## Remove 78400gw1 from this set, and replace this set in the list of congruent sets:

In [10]:
bad_s_ok = [bad_s[i] for i in 0,2,3]
bad_s_ok

['25921a1', '336973z1', '336973ba1']

In [11]:
ind = isom_sets_red.index(bad_s)
#ind = 165
isom_sets_red[ind] == bad_s

True

In [12]:
isom_sets_red[ind]=bad_s_ok

In [13]:
#print([(i,s) for i,s in enumerate(isom_sets_red) if '78400gw1' in s])
assert not  [s for s in isom_sets_red if '78400gw1' in s]

## Count the numbers of sets of irreducible and reducible congruent classes, and the siezes of these sets:

In [14]:
from collections import Counter
red_sizes = Counter()
irred_sizes = Counter()

In [48]:
for s in isom_sets_irred:
    irred_sizes[len(s)] += 1
irred_sizes

Counter({2: 36668, 3: 2594, 4: 492, 5: 12})

### So most irreducible sets have size 2 and the maximum size is 6

Find all the j-invariants of curves with congruences (irreducibles only):

In [17]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [18]:
irred_js = Set()
JS = irred_js_by_size = {2:Set(), 3:Set(), 4:Set(),5:Set()}
for s in isom_sets_irred:
    js = set_js(s)
    irred_js = irred_js + Set(js)
    irred_js_by_size[len(s)] += Set(js)

In [19]:
len(irred_js), [(n,len(irred_js_by_size[n])) for n in irred_js_by_size.keys()]

(10348, [(2, 9772), (3, 1313), (4, 320), (5, 20)])

In [20]:
len([j for j in irred_js_by_size[2] if not j in sum([irred_js_by_size[n] for n in [3,4,5]],Set())])

8924

In [21]:
[len([j for j in irred_js_by_size[3] if j in irred_js_by_size[n]]) for n in [4,5]]

[214, 10]

In [22]:
[len([j for j in irred_js_by_size[4] if j in irred_js_by_size[n]]) for n in [5]]

[12]

In [25]:
[n for n in [2,3,4] if 1728 in irred_js_by_size[n]]

[3]

In [31]:
sets3 = [s for s in isom_sets_irred if len(s)==3]
len(sets3)

1297

In [32]:
[s for s in sets3 if 1728 in set_js(s)]

[['576g1', '65088cu1', '65088cv1'],
 ['288e1', '32544bd1', '32544be1'],
 ['576f1', '65088cm1', '65088cn1'],
 ['288a1', '32544b1', '32544c1']]

In [34]:
test7iso(['288a1', '32544b1', '32544c1'])

[['288a1', '288a2', '32544b1', '32544c1'], []]

In [35]:
test7iso(['288e1', '32544bd1', '32544be1'])

[['288e1', '288e2', '32544bd1', '32544be1'], []]

In [36]:
test7iso(['576f1', '65088cm1', '65088cn1'])

[['576f1', '576f2', '65088cm1', '65088cn1'], []]

In [37]:
test7iso(['576g1', '65088cu1', '65088cv1'])

[['576g2', '576g1', '65088cu1', '65088cv1'], []]

In [28]:
jdict = {}
for s in isom_sets_irred:
    js = dict([(lab,class_js(lab)) for lab in s])
    for lab1 in s:
        E0 = EllipticCurve(lab1)
        other_j = sorted(list(sum([js[lab2] for lab2 in s if lab2!=lab1], Set())))
        for E in E0.isogeny_class():
            j = E.j_invariant()
            jdict[j] = other_j
        

In [30]:
len(jdict)

10348

In [16]:
len(isom_sets_irred)

19883

## Count sizes of reducible sets:

In [41]:
for s in isom_sets_red:
    red_sizes[len(s)] += 1
red_sizes  

Counter({2: 172, 3: 134, 4: 62, 5: 52, 7: 14, 10: 12, 6: 10, 8: 10, 9: 8, 19: 8, 11: 6, 16: 4, 18: 4, 32: 2, 35: 2, 76: 2, 13: 2, 14: 2, 45: 2, 36: 2})

### Reducible sets have various sizes up to 76

## Next we go through all the irreducible sets, taking all the curves in each isogeny class and partitioning them into 1 or 2 subsets such that curves in the same subset are symplectically isomorphic, while curves in different subsets are antisymplectically isomorphic.

### The code which does the work is in testcong.py

 all_c is a function defined in testcong.py which, given the label of one curve E in a 7-irreducible isogeny class,  return the list of all labels of curves in that class, partitioned    into 2 parts where those in the first part are m-isogenous to E           with legendre(m,7)=+1 and those in the second part (which may be   empty) are m-isogenous to E with legendre(m,7)=-1.
 
 The items in the lists output are pairs (curve label, isogeny degree)

In [42]:
[all_c(c) for c in isom_sets_red[0]]

[[('61347m1', 1), ('61347m2', 7)],
 [('306735l1', 1), ('306735l2', 7)],
 [('306735m1', 1), ('306735m2', 7)]]

In [43]:
[[all_c(c) for c in s] for s in isom_sets_irred[:10]]

[[[('15744e1', 1)], [('15744i1', 1), ('15744i2', 2)]],
 [[('330330gv1', 1)], [('330330hb1', 1)]],
 [[('20874m1', 1)], [('104370bt1', 1)]],
 [[('344025bc1', 1), ('344025bc2', 2)], [('344025bd1', 1)]],
 [[('229164e1', 1)], [('229164f1', 1)]],
 [[('13689d1', 1)], [('68445e1', 1)]],
 [[('163072k1', 1)], [('163072l1', 1)]],
 [[('1575f1', 1), ('1575f2', 2), ('1575f3', 4), ('1575f4', 4)],
  [('313425q1', 1)]],
 [[('28224gi1', 1), ('28224gi2', 3)], [('28224gl1', 1), ('28224gl2', 2)]],
 [[('79350cx1', 1)], [('79350cy1', 1)]]]

### Test one set, containing 2 isogeny classes:

In [44]:
isom_sets_irred[3]

['344025bc1', '344025bd1']

### The function test7iso() takes such a set of isogeny classes (of any size) and returns a list of length 2 lists of curve labels such that the curves in each sublist are symplectically 7-isogenous.  The second list may be empty if all the isomorphisms are symplectic, as in this example:

In [45]:
test7iso(isom_sets_irred[3])

[['344025bc1', '344025bc2', '344025bd1'], []]

### In this second example we see both symplectic and antisymplectic isomorphisms:

In [46]:
test7iso(isom_sets_irred[980])

[['100800gw1'], ['100800hc1', '100800hc2']]

### The preceding output means that '100800hc1' and '100800hc2' are symplectically isomorphic (in fact they are $2$-isogenous and $2$ is a quadratic non-residue) while '100800gw1' is anti-symplectically isomorphic to each.

In [47]:
all_c('100800hc1')

[('100800hc1', 1), ('100800hc2', 2)]

In [48]:
(test_isom_labels('100800hc1','100800hc2'),
test_isom_labels('100800gw1','100800hc1'),
test_isom_labels('100800gw1','100800hc2'))

(True, False, False)

In [49]:
(test_isom_labels('100800hc1','100800hc2', symplectic=False),
test_isom_labels('100800gw1','100800hc1', symplectic=False),
test_isom_labels('100800gw1','100800hc2', symplectic=False))

(False, True, True)

## Here follows the long run which takes several hours

In [50]:
def test7(lab1, lab2):
    E1 = EllipticCurve(lab1)
    E2 = EllipticCurve(lab2)
    res1 = test_isom(E1,E2,True)
    res2 = test_isom(E1,E2,False)
    if res1 and res2:
        print("Problem: curves {} and {} are both symplectically and anti-sympectically isomorphic!".format(lab1,lab2))
        return 0
    elif not (res1 or res2):
        print("Problem: curves {} and {} are neither symplectically nor anti-sympectically isomorphic!".format(lab1,lab2))
        return 0
    else:
        return +1 if res1 else -1

In [3]:
%%writefile t.py
nsets = len(isom_sets_irred)
irred_pairs_symp = []
irred_pairs_anti = []
irred_pairs_none = []
irred_pairs_bad = []
for i,s in enumerate(isom_sets_irred):
    print("Set #{}/{} with {} classes ({:.3f}%)".format(i+1,nsets,len(s),float(100.0*(i+1)/nsets)))
    #sys.stdout.write('\r')
    sys.stdout.flush()
    for i1,lab1 in enumerate(s):
        for i2, lab2 in enumerate(s):
            if i1<i2:
                try:
                    res = test7(lab1,lab2)
                    if res==+1:
                        sres = "symplectic"
                        irred_pairs_symp.append([lab1,lab2])
                    elif res==-1:
                        sres = "anti-symplectic"
                        irred_pairs_anti.append([lab1,lab2])
                    elif res==0:
                        sres = "neither"
                        irred_pairs_none.append([lab1,lab2])
                    else:
                        sres = "**********?????????*************"
                        irred_pairs_bad.append([lab1,lab2])
                    print("{} and {}: {}".format(lab1,lab2,sres))
                except:
                    print("error processing set number {}: {} and {}".format(i,lab1,lab2))

Overwriting t.py


In [ ]:
[len(t) for t in [irred_pairs_symp,irred_pairs_anti,irred_pairs_none,irred_pairs_bad]]

In [33]:
isom_sets_irred_symplectic2 = []
nsets = len(isom_sets_irred[:20])
for i,s in enumerate(isom_sets_irred[:20]):
    if i%20==19:
        print("Set #{}/{} ({:.3f}%)".format(i+1,nsets,float(100.0*(i+1)/nsets)))
    sys.stdout.write('\r')
    sys.stdout.flush()
    try:
        res = test7iso(s)
        isom_sets_irred_symplectic2.append(res)
        #print(res)
    except:
        print("error processing set number {}: {}".format(i,s))

Set #20/20 (100.000%)


In [22]:
[(i,s1,s2) for i,s1,s2 in zip(range(len(isom_sets_irred_symplectic2)), isom_sets_irred_symplectic, isom_sets_irred_symplectic2) if s1!=s2]

NameError: name 'isom_sets_irred_symplectic2' is not defined

In [25]:
len(isom_sets_irred_symplectic2)

19883

In [26]:
out = open("mod7irreducible_congruences.txt",'w')
for ss in isom_sets_irred_symplectic2:
    out.write(str(ss))
    out.write("\n")
out.close()    

In [27]:
isom_sets_irred_anti = [ss for ss in isom_sets_irred_symplectic2 if ss[1]]
len(isom_sets_irred_anti), len(isom_sets_irred_symplectic2)-len(isom_sets_irred_anti)

(7489, 12394)

In [25]:
from sage.databases.cremona import parse_cremona_label
parse_cremona_label("11a1")

(11, 'a', 1)

In [26]:
def class_label(label):
    return "".join([str(s) for s in parse_cremona_label(label)[:2]])

In [27]:
class_label("11a1")

'11a'

In [28]:
def non_triv(ss):
    classes = [[class_label(s) for s in si] for si in ss]
    return not any(c in classes[1] for c in classes[0])

In [32]:
isom_sets_irred_nontriv = [ss for ss in isom_sets_irred_anti if non_triv(ss)]
len(isom_sets_irred_nontriv)

5092

In [33]:
out = open("mod7_antipairs.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_nontriv[:-1]:
    print(s[0],s[1])
    out.write('["{}", "{}"],\\\n'.format(s[0][0],s[1][0]))
s = isom_sets_irred_nontriv[-1]
out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

(['18326z1'], ['238238ba1'])
(['100800gw1'], ['100800hc1', '100800hc2'])
(['1178b1'], ['133114h1'])
(['1872r1', '1872r2', '1872r3', '1872r4'], ['87984bs1', '91728ft1'])
(['159936jh1'], ['159936jq1'])
(['204309p1'], ['364203v1'])
(['66240en1'], ['66240es1'])
(['9360bk1'], ['271440ch1'])
(['6534f1'], ['45738v1', '280962z1'])
(['17974i1'], ['17974j1'])
(['24780a1'], ['338660f1'])
(['28224et1'], ['28224gc1', '28224gc2'])
(['105105bb1'], ['105105bc1'])
(['87210w1'], ['87210x1'])
(['39984ct1'], ['39984do1'])
(['94640df1'], ['283920di1'])
(['3600br2', '3600br1'], ['46800ev1', '61200gs1'])
(['176400dm1'], ['176400fq1'])
(['95370dr1'], ['95370ds1'])
(['158025bh1'], ['158025bi1'])
(['144150bw1'], ['144150cn1'])
(['41292g1'], ['41292h1'])
(['19881g1'], ['139167i1'])
(['161994u1'], ['161994bg1'])
(['18928be1'], ['56784dc1'])
(['2704l1'], ['8112bf1'])
(['9675j1'], ['396675q1'])
(['3042a2', '3042a1'], ['57798d1'])
(['280476bd1'], ['280476be1'])
(['9702o1', '9702o2'], ['300762cn1'])
(['22050g1'], ['2

(['1950h1'], ['142350bp1'])
(['322896h1', '322896h2'], ['322896i2', '322896i1'])
(['122304gr1'], ['122304ij4', '122304ij2', '122304ij3', '122304ij1'])
(['10150d1'], ['131950ce1'])
(['198a4', '198a1', '198a2', '198a3'], ['4554o1', '110286j1'])
(['34496cd1'], ['103488gz1'])
(['3780c1'], ['223020bk1'])
(['275373l1'], ['275373m1'])
(['163350cx1'], ['163350ee1'])
(['310464ew1'], ['310464fb2', '310464fb1'])
(['16560bl1'], ['182160cr1'])
(['204490ch1'], ['204490cm1'])
(['35772a1'], ['276420f1'])
(['187425fk1'], ['253575fk1'])
(['17550cb1'], ['122850cr1'])
(['286650v1'], ['286650gk1'])
(['61446cc1'], ['102410bs1'])
(['318270p1'], ['318270q1'])
(['127050br1'], ['127050bz1'])
(['8820n2', '8820n1'], ['8820p1'])
(['101454c1'], ['150810l1'])
(['353934bg1', '353934bg2'], ['353934bh1'])
(['14514n1'], ['237062u1'])
(['78400a1'], ['78400s1'])
(['136710gm1'], ['136710gr1'])
(['302400lt1'], ['302400sa1'])
(['17424br1'], ['87120el1'])
(['132940b1'], ['398820bn1'])
(['168432i1'], ['168432j1'])
(['141120iz1

(['91728fs1'], ['91728fw2', '91728fw1'])
(['2800bf1'], ['8400bw1', '8400bw2'])
(['152460x1'], ['152460z1', '152460z2'])
(['344988f1'], ['344988h1'])
(['134946cu1'], ['134946cx1'])
(['1360e2', '1360e1'], ['131920l1'])
(['11466g1', '11466g2'], ['217854en1'])
(['368f1', '54096bn1'], ['18768o1'])
(['2160u1', '153360p1'], ['41040br1'])
(['34914a1'], ['174570da1'])
(['19008br1'], ['95040gw1'])
(['3612g1'], ['299796i1'])
(['6084p1', '6084p2'], ['298116s1'])
(['20286c1'], ['385434ck1'])
(['225302cx1'], ['225302cy1'])
(['154154m1'], ['154154ba1'])
(['4554bh1'], ['159390e2', '159390e1'])
(['282150cx1'], ['282150fe1'])
(['52800fy1'], ['123200ga1'])
(['35490cf1', '35490cf2'], ['291018ch1'])
(['31050bi1'], ['341550eb1'])
(['37926g1', '37926p1'], ['37926r1'])
(['63296o1', '217408e1'], ['244928d1'])
(['2975a1'], ['235025a1'])
(['36300bl1'], ['36300bm1'])
(['170352c1'], ['170352cu1'])
(['282576k3', '282576k2', '282576k1', '282576k6', '282576k5', '282576k4'], ['282576o1'])
(['1650e1', '51150t1'], ['169

In [34]:
out = open("mod7_antipairs_all.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_symplectic2:
    for s0 in s[0]:
        for s1 in s[1]:
            out.write('["{}", "{}"],\\\n'.format(s0,s1))
#s = isom_sets_irred_symplectic2[-1]
#out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

In [15]:
%run XE7

In [16]:
R2.<t> = QQ[]

In [104]:
j0sets = [s for s in isom_sets_irred if any([EllipticCurve(lab).j_invariant()==0 for lab in s])]

In [248]:
asq_sets = [isom_sets_irred[i] for i in [22,27,43,63,70,71,76,84]]
for s in asq_sets:
    print(test7iso(s))

[['9016m1', '171304h1', '333592j1'], []]
[['5550bk1', '5550bm4', '5550bm1', '5550bm3', '5550bm2'], []]
[['33856q1', '169280bl1'], []]
[['9800f1', '9800g1'], []]
[['82810b1', '82810g1'], []]
[['40768a1', '40768p1'], []]
[['174048a1', '174048p1'], []]
[['290325bt1', '290325bu1'], []]


## Testing isomorphisms among sets isogeny classes which are reducible and mutually isomorphic up to semisimplification

In [51]:
len(isom_sets_red)

255

In [52]:
%run testcong.py

In [53]:
isom_sets_red_complete = []
for i, ss in enumerate(isom_sets_red):
    S1, S2 = mod_p_iso_red_set(ss,7,0)
    #print(i, S1, S2)
    nontriv = [s for s in S1+S2 if len(s)>1]
    if nontriv:
        print(nontriv)
        isom_sets_red_complete += nontriv
        

[['40432o1', '63536q1']]
[['4050f3', '125550m2']]
[['8450t1', '177450di1', '278850gb1']]
[['78400go2', '235200bby2']]
[['74529w2', '74529x3'], ['74529m2', '74529x1']]
[['5054a1', '7942g1'], ['25270d1', '299630c1'], ['3610b2', '107578e2']]
[['11025y1', '143325ev2'], ['11025y3', '143325eq2']]
[['13754j1', '288834cc1'], ['155526p2', '259210bl2'], ['155526p1', '358662y2']]
[['10816y1', '54080by1']]
[['40898bg1', '204490ba1']]
[['8450n1', '8450o1'], ['8450o2', '312650ce2']]
[['12675y1', '392925v1'], ['12675y2', '257725a2']]
[['76050br1', '76050bs1']]
[['9408bp2', '15680dg2'], ['40768db1', '122304fe1']]
[['126350c1', '198550ck1']]
[['176400hn2', '176400ic2']]
[['78400dk2', '235200pp2']]
[['19350by1', '212850v1'], ['22050eb2', '22050ev2'], ['18450bu1', '129150cr1'], ['5850bq1', '40950ea1', '64350du1', '286650ou2'], ['22050ev1', '50850bd2']]
[['784h1', '10192m2', '141904s1'], ['784h3', '10192bd2', '141904t2']]
[['351310k2', '373498j2']]
[['146205e2', '321651f2']]
[['46800dr1', '327600nn1'], ['

[['162c1', '17334f1', '293706x2'], ['162c3', '5022d2'], ['167994i2', '184518l2']]
[['1296e1', '138672m1'], ['1296e3', '40176p2']]
[['16562d1', '82810bc1']]
[['63504y1', '317520hf2']]
[['161728j1', '254144cc1']]
[['16562r1', '49686i1', '248430ci2'], ['49686c2', '82810ba2']]
[['6480l1', '110160by1'], ['6480l2', '14256m2', '63504bm1']]
[['106134u2', '176890cn2']]
[['28224fg1', '366912jb2'], ['28224fg3', '366912ja2']]
[['16562be1', '82810bu1']]
[['149093d2', '356307j2']]
[['1296k1', '138672f1'], ['1296k3', '40176r2']]
[['161728bk1', '254144ce1']]
[['22050dr2', '22050dx2'], ['286650je1', '286650ol1', '286650om2']]
[['161728bj1', '254144cf1']]
[['25920cw2', '57024cx2', '254016do1']]
[['2704e1', '13520w1'], ['2704e2', '100048k2']]
[['8112bd2', '164944b2']]
[['31850q1', '95550da2', '95550ee1'], ['100450u1', '301350cc1'], ['2450c2', '7350u2']]
[['35378k1', '389158bg1']]
[['14161a1', '184093l2'], ['14161a3', '184093k2']]
[['132496br1', '132496bs2'], ['132496bp2', '132496br3']]
[['3042l1', '15210

In [54]:
len(isom_sets_red_complete)

337

In [55]:
[(n, len([s for s in isom_sets_red_complete if len(s)==n])) for n in [2,3,4]]

[(2, 276), (3, 52), (4, 9)]

In [56]:
Set(len(s) for s in isom_sets_red_complete)

{2, 3, 4}

In [58]:
[[EllipticCurve(si).j_invariant() for si in s] for s in isom_sets_red_complete if len(s)==4]

[[-2146689/1664,
  40251338884511/2997011332224,
  -21293376668673906679951249/26211168887701209984,
  -131425499875625809/4658135040000000],
 [-2146689/1664,
  40251338884511/2997011332224,
  -21293376668673906679951249/26211168887701209984,
  -131425499875625809/4658135040000000],
 [-2146689/1664,
  40251338884511/2997011332224,
  -21293376668673906679951249/26211168887701209984,
  -131425499875625809/4658135040000000],
 [-2146689/1664,
  40251338884511/2997011332224,
  -131425499875625809/4658135040000000,
  -21293376668673906679951249/26211168887701209984],
 [-2146689/1664,
  40251338884511/2997011332224,
  -21293376668673906679951249/26211168887701209984,
  -131425499875625809/4658135040000000],
 [-2146689/1664,
  40251338884511/2997011332224,
  -21293376668673906679951249/26211168887701209984,
  -131425499875625809/4658135040000000],
 [-2146689/1664,
  40251338884511/2997011332224,
  -131425499875625809/4658135040000000,
  -21293376668673906679951249/26211168887701209984],
 [-214

In [62]:
len(Set([Set([EllipticCurve(si).j_invariant() for si in s]) for s in isom_sets_red_complete if len(s)==3]))

10

In [63]:
len(Set([Set([EllipticCurve(si).j_invariant() for si in s]) for s in isom_sets_red_complete if len(s)==2]))

39

In [65]:
isom_sets_red_split = []
for i,ss in enumerate(isom_sets_red_complete):
    E1 = ss[0]
    symps = [E1]
    antis = []
    for E2 in ss[1:]:
        res = test_isom_labels(E1,E2, False)
        print("The isomorphism between {} and {} is {}".format(E1,E2,"symplectic" if res else "antisyplectic"))
        (symps if res else antis).append(E2)
    isom_sets_red_split.append([symps,antis])    
    print("{}: {}={}+{}".format(i,len(ss),len(symps),len(antis)))

The isomorphism between 40432o1 and 63536q1 is symplectic
0: 2=2+0
The isomorphism between 4050f3 and 125550m2 is symplectic
1: 2=2+0
The isomorphism between 8450t1 and 177450di1 is antisyplectic
The isomorphism between 8450t1 and 278850gb1 is symplectic
2: 3=2+1
The isomorphism between 78400go2 and 235200bby2 is symplectic
3: 2=2+0
The isomorphism between 74529w2 and 74529x3 is antisyplectic
4: 2=1+1
The isomorphism between 74529m2 and 74529x1 is antisyplectic
5: 2=1+1
The isomorphism between 5054a1 and 7942g1 is symplectic
6: 2=2+0
The isomorphism between 25270d1 and 299630c1 is symplectic
7: 2=2+0
The isomorphism between 3610b2 and 107578e2 is symplectic
8: 2=2+0
The isomorphism between 11025y1 and 143325ev2 is antisyplectic
9: 2=1+1
The isomorphism between 11025y3 and 143325eq2 is antisyplectic
10: 2=1+1
The isomorphism between 13754j1 and 288834cc1 is antisyplectic
11: 2=1+1
The isomorphism between 155526p2 and 259210bl2 is symplectic
12: 2=2+0
The isomorphism between 155526p1 and

The isomorphism between 6480z2 and 63504cr1 is antisyplectic
99: 3=1+2
The isomorphism between 45486n1 and 71478i1 is symplectic
100: 2=2+0
The isomorphism between 2704d1 and 13520o1 is symplectic
101: 2=2+0
The isomorphism between 2704d2 and 100048e2 is symplectic
102: 2=2+0
The isomorphism between 165886k1 and 355470bj1 is antisyplectic
103: 2=1+1
The isomorphism between 84966ct2 and 141610bk2 is symplectic
104: 2=2+0
The isomorphism between 7514i1 and 157794bc1 is antisyplectic
The isomorphism between 7514i1 and 247962y1 is symplectic
105: 3=2+1
The isomorphism between 84966ct1 and 195942p2 is antisyplectic
106: 2=1+1
The isomorphism between 891d1 and 223641h1 is antisyplectic
107: 2=1+1
The isomorphism between 405c1 and 6885c1 is antisyplectic
The isomorphism between 405c1 and 284715m1 is antisyplectic
108: 3=1+2
The isomorphism between 405c2 and 891d2 is antisyplectic
The isomorphism between 405c2 and 3969c1 is antisyplectic
109: 3=1+2
The isomorphism between 149058bq1 and 149058b

The isomorphism between 19600cb3 and 254800eh2 is antisyplectic
199: 2=1+1
The isomorphism between 155682f2 and 155682i3 is symplectic
200: 2=2+0
The isomorphism between 5184a3 and 160704bd2 is symplectic
201: 2=2+0
The isomorphism between 25168bp1 and 75504bt1 is symplectic
202: 2=2+0
The isomorphism between 84966dm2 and 141610c2 is symplectic
203: 2=2+0
The isomorphism between 832j1 and 17472cs1 is antisyplectic
The isomorphism between 832j1 and 27456cj1 is symplectic
204: 3=2+1
The isomorphism between 9408ct2 and 15680cw2 is symplectic
205: 2=2+0
The isomorphism between 8256bp1 and 151360b1 is antisyplectic
206: 2=1+1
The isomorphism between 18368u1 and 39360ch1 is antisyplectic
207: 2=1+1
The isomorphism between 9408ct1 and 21696by2 is antisyplectic
208: 2=1+1
The isomorphism between 2025f1 and 34425q1 is antisyplectic
209: 2=1+1
The isomorphism between 2025f2 and 22275j2 is antisyplectic
The isomorphism between 2025f2 and 99225bi1 is antisyplectic
210: 3=1+2
The isomorphism betwee

In [66]:
assert len(isom_sets_red_split)==len(isom_sets_red_complete)

In [67]:
set([tuple([len(s) for s in ss]) for ss in isom_sets_red_split])

{(1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (3, 0)}

In [36]:
ofile_s = open("mod7_red_symp_pairs.m", 'w')
ofile_a = open("mod7_red_anti_pairs.m", 'w')
ofile_s.write('mod7_red_symp_pairs := [\\\n')
ofile_a.write('mod7_red_anti_pairs := [\\\n')
first_a = first_s = True
for ss in isom_sets_red_split:
    for s in ss:
        if s:
            s0 = s[0]
            for s1 in s[1:]:
                if not first_s:
                    ofile_s.write(", ")
                else:
                    first_s = False
                ofile_s.write('["{}", "{}"]\\\n'.format(s0,s1))
    s0 = ss[0][0]
    for s1 in ss[1]:
        if not first_a:
            ofile_a.write(", ")
        else:
            first_a = False
        ofile_a.write('["{}", "{}"]\\\n'.format(s0,s1))
ofile_s.write('];\n')
ofile_a.write('];\n')
ofile_s.close()
ofile_a.close()

In [44]:
t = polygen(QQ,'t')
E0 = KH0(t)
E1 = KH1(t)
E2 = KH2(t)

In [45]:
E0.discriminant().factor()

(-338688) * t * (t + 49/27)

In [36]:
E1.discriminant().factor()

(-432) * (t + 49/27) * t^2 * (t + 3)^7

In [37]:
E2.discriminant().factor()

(-122009559759792) * t * (t + 49/27)^2 * (t - 32/27)^7

In [43]:
E1.discriminant().gcd(E1.c6())

1

In [72]:
Ea0 = KH0(-3)
Ea0.label(), Ea0.j_invariant()

('294b1', -2401/6)

In [65]:
Ea2 = KH2(-3)
Ea2.label(), Ea2.j_invariant()

('678d2', -39934705050538129/2823126576537804)

In [66]:
Eb0 = KH0(32/27)
Eb0.label(), Eb0.j_invariant()

('2352l1', -2401/6)

In [67]:
Eb1 = KH1(32/27)
Eb1.label(), Eb1.j_invariant()

('5424l2', -39934705050538129/2823126576537804)

In [68]:
Eb0.is_quadratic_twist(Ea0)

-1

In [70]:
E0.j_invariant().factor()

(-38416/27) * t^-1 * (t + 49/27)^-1

### Now we reduce to a smaller sets  such that every congruent set is a twist of one in the smaller set

In [28]:
j_pairs_irred = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in isom_sets_irred]
j_sets_irred = list(Set(j_pairs_irred))
print("Out of {} irreducible sets, there are only {} distinct sets of j-invariants".format(len(j_pairs_irred),len(j_sets_irred)))

Out of 19883 irreducible sets, there are only 4939 distinct sets of j-invariants


In [29]:
j_pairs_red = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in isom_sets_red]
j_sets_red = list(Set(j_pairs_red))
print("Out of {} reducible sets, there are only {} distinct sets of j-invariants".format(len(j_pairs_red),len(j_sets_red)))

Out of 255 reducible sets, there are only 102 distinct sets of j-invariants


In [30]:
19883+255

20138

## Find all j-invariants of all curves (not just one per isogeny class) with congruences.  First, irreducible ones:

In [35]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [36]:
class_js('11a1')

{-122023936/161051, -4096/11, -52893159101157376/11}

In [37]:
set_js(isom_sets_irred[0])

{-12260311912687962747488/52751561202729, -6340969184/123, 392331196600562528112256/7263027}

In [38]:
all_irred_js = sum([set_js(s) for s in isom_sets_irred], Set())

In [39]:
all_red_js_ss = sum([set_js(s) for s in isom_sets_red], Set())

In [40]:
all_red_js = sum([set_js(s) for s in isom_sets_red_complete], Set())

NameError: name 'isom_sets_red_complete' is not defined

In [ ]:
len(all_irred_js), len(all_red_js), len(all_red_js_ss)

In [35]:
E = EllipticCurve(j=0)

In [37]:
E.reducible_primes?

In [39]:
E.label()

'27a3'

In [44]:
[j for j in sum(j_sets_red, Set()) if j in cm_j_invariants(QQ)]

[-3375]

In [45]:
cm_j_invariants_and_orders(QQ)

[(-3, 3, -12288000),
 (-3, 2, 54000),
 (-3, 1, 0),
 (-4, 2, 287496),
 (-4, 1, 1728),
 (-7, 2, 16581375),
 (-7, 1, -3375),
 (-8, 1, 8000),
 (-11, 1, -32768),
 (-19, 1, -884736),
 (-43, 1, -884736000),
 (-67, 1, -147197952000),
 (-163, 1, -262537412640768000)]

In [76]:
len(isom_sets_irred)

19883

In [49]:
E = EllipticCurve(j=-32768)

In [50]:
E.label()

'121b1'

In [52]:
E.quadratic_twist(-11).label()

'121b2'

In [55]:
[s for s in isom_sets_irred if '9225a1' in s]

[['225a1', '9225a1', '9225e1', '11025c1']]

In [62]:
test_isom_labels('9225a1','11025c1', symplectic=False)

False

In [88]:
E = EllipticCurve('225a1')

In [69]:
EllipticCurve('225a1').is_quadratic_twist(EllipticCurve('11025c1'))

0

In [82]:
EllipticCurve('225a1').quadratic_twist(-3).label()

'225a2'

In [87]:
test_isom_labels('225a1','225a2',symplectic=False)

True

In [93]:
EllipticCurve('9225c1').j_invariant()

-1032125355/41

In [89]:
E.short_weierstrass_model()

Elliptic Curve defined by y^2 = x^3 + 80 over Rational Field

In [92]:
EllipticCurve([0,0,0,0,-28/80]).minimal_model().label()

'11025c1'

In [47]:
[test_isom(EllipticCurve([1,0]), EllipticCurve([(-4)*(-7)^k,0]), False) for k in range(4)]

[False, False, False, False]

In [40]:
test_isom?

In [87]:
sets2 = [s for s in isom_sets_irred if len(s)==2]
sets3 = [s for s in isom_sets_irred if len(s)==3]
sets4 = [s for s in isom_sets_irred if len(s)==4]
sets5 = [s for s in isom_sets_irred if len(s)==5]
len(sets2),len(sets3),len(sets4), len(sets5)

(18334, 1297, 246, 6)

In [78]:
def test4(s):
    js = [class_js(e) for e in s]
    #print(js)
    for i1 in range(len(js)):
        js1 = js[i1]
        for i2 in range(i1+1,len(js)):
            js2 = js[i2]
            #print(js1.intersection(js2))
            if len(js1.intersection(js2))>0: 
                print(s, js1, js2)

In [79]:
for s in sets4:
    test4(s)

(['225b1', '9225i1', '9225l1', '11025k1'], {0}, {0})
(['225b1', '9225i1', '9225l1', '11025k1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['3600be1', '147600dd1', '147600de1', '176400ip1'], {0}, {0})
(['3600be1', '147600dd1', '147600de1', '176400ip1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['3600bc1', '147600dp1', '147600dq1', '176400ke1'], {0}, {0})
(['3600bc1', '147600dp1', '147600dq1', '176400ke1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['225a1', '9225a1', '9225e1', '11025c1'], {0}, {0})
(['225a1', '9225a1', '9225e1', '11025c1'], {2813708206080/194754273881}, {2813708206080/194754273881})


In [68]:
E1 = EllipticCurve('225b1')
E2 = EllipticCurve('9225i1')
E3 = EllipticCurve('9225l1')
E4 = EllipticCurve('11025k1')
for E in [E1,E2,E3,E4]:
    print("{}: {}".format(E.label(), E.j_invariant()))
    print(E.galois_representation().image_type(7))

225b1: 0
The image is contained in the normalizer of a split Cartan group. (cm)
9225i1: 2813708206080/194754273881
The image is contained in the normalizer of a split Cartan group.
9225l1: 2813708206080/194754273881
The image is contained in the normalizer of a split Cartan group.
11025k1: 0


NotImplementedError: image_type is not implemented for cm-curves at bad prime.

In [72]:
E1.is_sextic_twist(E4)

-7/25

In [75]:
E1.short_weierstrass_model(), E4.short_weierstrass_model()

(Elliptic Curve defined by y^2 = x^3 + 10000 over Rational Field,
 Elliptic Curve defined by y^2 = x^3 - 2800 over Rational Field)

In [77]:
(-28).factor()

-1 * 2^2 * 7

In [83]:
for s in sets3:
    test4(s)

(['576g1', '65088cu1', '65088cv1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['288e1', '32544bd1', '32544be1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['576f1', '65088cm1', '65088cn1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['378225x1', '378225bm1', '378225bn1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['288a1', '32544b1', '32544c1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['378225bd1', '378225be1', '378225bq1'], {2813708206080/194754273881}, {2813708206080/194754273881})


In [85]:
E1 = EllipticCurve('378225bd1')
E2 = EllipticCurve('378225be1')
E3 = EllipticCurve('378225bq1')
[E.j_invariant() for E in [E1,E2,E3]]

[2813708206080/194754273881, 2813708206080/194754273881, 0]

In [86]:
E1.is_quadratic_twist(E2)

-3

In [88]:
for s in sets2: test4(s)

(['163072k1', '163072l1'], {-69832731032000/62748517}, {-69832731032000/62748517})
(['33075b1', '33075i1'], {0}, {0})
(['2700t1', '132300n1'], {0}, {0})
(['379456gs1', '379456gt1'], {16807000}, {16807000})
(['160083bs1', '160083bv1'], {0}, {0})
(['243a1', '11907g1'], {0}, {0})
(['151686m1', '151686n1'], {194967154845375/128}, {194967154845375/128})
(['44100u1', '44100y1'], {0}, {0})
(['4356a1', '213444dt1'], {0}, {0})
(['11907o1', '11907t1'], {0}, {0})
(['84672cr1', '84672cz1'], {0}, {0, -12288000})
(['297675ce1', '297675ct1'], {0}, {0})
(['176400kf1', '176400ky1'], {0, 54000}, {0})
(['160083bt1', '160083bw1'], {0, -12288000}, {0})
(['28224a1', '28224i1'], {0}, {0, 54000})
(['379456dd1', '379456de1'], {16807000}, {16807000})
(['298116bx1', '298116ce1'], {0}, {0, 54000})
(['3267d1', '160083bb1'], {0, -12288000}, {0})
(['2700l1', '132300bt1'], {0}, {0})
(['7056bb1', '7056bf1'], {0}, {0, 54000})
(['21168bp1', '21168bz1'], {0}, {0})
(['27a1', '1323m1'], {0, -12288000}, {0})
(['576a1', '282

(['298116br1', '298116cb1'], {0}, {0})
(['176400x1', '176400cx1'], {2268945/128}, {2268945/128})
(['298116bp1', '298116bv1'], {0}, {0})
(['132300du1', '132300es1'], {0}, {0})
(['190512dj1', '190512dn1'], {0}, {0})
(['382347bp1', '382347cb1'], {0}, {0})
(['176400iy1', '176400je1'], {0}, {0})
(['2450ba1', '2450bd1'], {2268945/128}, {2268945/128})
(['132300m1', '132300y1'], {0}, {0})
(['44100t1', '44100v1'], {0}, {0})
(['7803c1', '382347cd1'], {0}, {0})
(['190512df1', '190512du1'], {0}, {0})
(['3888p1', '190512v1'], {0}, {0})
(['1728v1', '84672iy1'], {0, -12288000}, {0})
(['5292h1', '5292j1'], {0}, {0})
(['47628a1', '47628j1'], {0}, {0})
(['11025j1', '11025l1'], {0}, {0})
(['189728bg1', '189728bh1'], {16807000}, {16807000})
(['6075a1', '297675cs1'], {0}, {0})
(['3267a1', '160083bq1'], {0}, {0})
(['432b1', '21168cz1'], {0}, {0})
(['297675ca1', '297675cn1'], {0}, {0})
(['28224cw1', '28224di1'], {0}, {0, 54000})
(['127449e1', '127449m1'], {0}, {0})
(['159201w1', '159201bf1'], {0}, {0})
(['13